In [1]:
import numpy as np
import sys, os

In [50]:
class Player:
    
    def __init__(self, first, last, pos, team, pwr, cnt, spd, fld):
        
        # name and position
        self.first = first 
        self.last = last
        self.pos = pos
        self.team = team
        
        # skills
        self.pwr = pwr
        self.cnt = cnt
        self.spd = spd
        self.fld = fld
        
        # stat counts
        self.singles = 0
        self.doubles = 0
        self.triples = 0
        self.walks = 0
        self.atbats = 0
        self.runs = 0
        self.rbis = 0
        self.hmrs = 0
        
        # stat calculations
        self.hits = self.singles + self.doubles + self.triples + self.hmrs
        if self.atbats == 0:
            self.avg = 0.0
        else:
            self.avg = np.round(self.hits/(self.atbats - self.walks), 3)
            
    def print_skills(self):
        return f'pwr={self.pwr} cnt={self.cnt} spd={self.spd} fld={self.fld}'
    
    def hit_list(self):
        return np.array([self.pwr, self.cnt, self.spd])
    
    def print_stats(self):
        
        # stat calculations
        self.hits = self.singles + self.doubles + self.triples + self.hmrs
        if self.atbats == 0:
            self.avg = 0.0
        else:
            self.avg = np.round(self.hits/self.atbats, 3)
            
        return f'hits={self.hits} walks={self.walks} atbat={self.atbats} avg={self.avg} rbi={self.rbis} hmr={self.hmrs}'

    
class Team:
    
    def __init__(self, county, mascot, roster):
        
        # name info
        self.county = county
        self.mascot = mascot
        self.roster = roster

        # stats 
        self.win = 0
        self.loss = 0
        
    def print_record(self):
        return f'{self.win}-{self.loss}'

    
class Game:
    
    max_outs = 3
    max_stks = 3
    max_blls = 4
    max_inning = 9
    empty_diamond = [None, None, None]
    
    # dict corresponds to corefficients multiplied by [pwr, cnt, spd] to creat cum probability
    # probabilites correspond to a results after any given pitch
    action_dict = {'S':5*np.array([.6, 1.5, 1]),
                'D':2*np.array([.8, 1.3, 1.2]), 
                'T':1*np.array([1, 1, 1.5]),
                'H':1*np.array([2, .8, .8]),
                'O':12*np.array([1.3, 1.3, .8]),
                'K':8*np.array([1.5, .8, 1]),
                'B':2*np.array([1.5, 1.2, .7])}
    
    def __init__(self, h_team, a_team):
        
        self.rosters = [h_team.roster, a_team.roster]
    
        self.inning = 1
        
        self.atbat = [0, 0]
        self.hits = [0, 0]
        self.runs = [0, 0] 
        self.errs = [0, 0]
    
    # for now treat this as a full at bat resulting in an action
    def throw_pitch(self, team_num, p):
        
        p_hit_list = p.hit_list() 
        
        # calculate action probabilities:
        p_dict = {}
        action_norm = 0
        for key in Game.action_dict:
            
            # take inner product of skill list with action vals
            action_weight = np.dot(Game.action_dict[key], p_hit_list)
            action_norm += action_weight
            p_dict[key] = action_weight
        
        # makes weighted action choice
        action = np.random.choice(list(p_dict.keys()), p=[w/action_norm for w in list(p_dict.values())])
        print(action)
        
        return action
    
    # updates diamond and runs base runners advanced by basehits
    # prob = probability of advancing an extra base
    def advance_runners(self, team_num, bases, p_i):
        
        new_diamond = [None, None, None]
        batter = self.rosters[team_num][p_i]
        
        # assign advance probability based on speed of leading runner
        prob = 0
        for x in reversed(self.diamond):
            if x != None:
                
                lead_runner = self.rosters[team_num][x]
                prob = lead_runner.spd/10
                break
        
        # advance based on prob
        advance = np.random.choice([True, False], p=[prob, 1-prob])
        
        # score all runners on bases
        if (bases == 2 and advance) or bases == 3:
            runs = (3 - self.diamond[:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
        
        # move runner from 1st to 3rd
        elif (bases == 1 and advance) or (bases == 2 and not advance):
            runs = (2 - self.diamond[1:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            new_diamond[2] = self.diamond[0]

        # advance runners one base
        elif (bases == 0 and advance) or (bases == 1 and not advance):  
            runs = (1 - self.diamond[2:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            new_diamond[2] = self.diamond[1]
            new_diamond[1] = self.diamond[0]
        
        # no advance means only a change if force
        elif bases == 0 and not advance:
            new_diamond = self.diamond
            
        # place batter on appropriate base
        if bases > 0:
            new_diamond[bases-1] = p_i
            
        self.diamond = new_diamond
    
    # takes in action and makes appropriate game updates
    def action_update(self, team_num, action, p_i):
        
        batter = self.rosters[team_num][p_i]
        batter.atbats += 1
    
        if action == 'K':
            self.outs += 1
            
        elif action == 'O':
            self.outs += 1
            
            # runners may advance if less than 3 outs, remove runner from diamond
            if self.outs < Game.max_outs:
                Game.advance_runners(self, team_num, 0, p_i)
                
        elif action == 'B':
            new_diamond = [None, None, None]
            batter.atbats -= 1

            if self.diamond[0] is not None:
                if self.diamond[1] is not None:
                    if self.diamond[2] is not None:
                        
                        self.runs[team_num] += 1
                        batter.rbis += 1
                        
                    new_diamond[2] = self.diamond[1]
                new_diamond[1] = self.diamond[0]
            new_diamond[0] = p_i
            
            self.diamond = new_diamond
            batter.walks += 1
            
        elif action == 'S':
            Game.advance_runners(self, team_num, 1, p_i)
            batter.singles += 1
        
        elif action == 'D':
            Game.advance_runners(self, team_num, 2, p_i)
            batter.doubles += 1
            
        elif action == 'T':
            Game.advance_runners(self, team_num, 3, p_i)
            batter.triples += 1
            
        elif action == 'H':
            runs = 1 + (3 - self.diamond[:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            batter.hmrs += 1
            self.diamond = Game.empty_diamond
            
    def play_game(self, verbose=1):
        
        # iterate through each inning
        while self.inning < Game.max_inning+1 or self.runs[0] == self.runs[1]:
            
            if verbose:
                print('inning = ', self.inning)
            
            # play inning for away team
            for team_num in [1, 0]:
                self.outs = 0
                self.diamond = Game.empty_diamond[:]
                while self.outs < Game.max_outs:

                    if verbose:
                        print(self.diamond)
                    
                    # player index -- who is at bat
                    p_i = self.atbat[team_num]
                    p = self.rosters[team_num][p_i]

                    action = Game.throw_pitch(self, team_num, p)
                    Game.action_update(self, team_num, action, p_i)

                    # advance order in lineup by one
                    self.atbat[team_num] = (self.atbat[team_num] + 1)%9

                    #print('outs = ', self.outs)
                if team_num == 1: print('switch')
    
            if verbose:
                print(self.runs)
                
            self.inning += 1
        
        if self.runs[0] > self.runs[1]:
            return True, self.rosters
        else:
            return False, self.rosters
    


In [52]:
pos_list = ['C', '1B', '2B', '3B', 'SS', 'DH', 'LF', 'RH', 'CF']
county_list = ['Litchfield', 'Hartford', 'Tolland', 'Windham', 
               'Fairfield', 'New Haven', 'Middlesex', 'New London']
mascot_list = ['Leopards', 'Warriors', 'Braves', 'Bandits',
              'Founders', 'Sharks', 'Rebels', 'Whalers']

In [53]:
def assign_skills():
    
    norm_var = 2.5
    
    # assign power
    pwr = np.round(np.random.uniform(1, 9), 0)
    
    # assign contact
    cnt_mean = 6.2 - pwr/4
    cnt = np.round(np.random.normal(cnt_mean, norm_var), 0)
    
    if cnt < 1:
        cnt = 1
    elif cnt > 9:
        cnt = 9
        
    # assign speed
    spd_mean = 5.2 - pwr/3 + cnt/4
    spd = np.round(np.random.normal(spd_mean, norm_var), 0)
    
    if spd < 1:
        spd = 1
    elif spd > 9:
        spd = 9
        
    # assign fld
    fld_mean = 7 - (pwr + cnt + spd)/7
    fld = np.round(np.random.normal(fld_mean, norm_var), 0)
    
    if fld < 1:
        fld = 1
    elif fld > 9:
        fld = 9
    
    skill_list = [pwr, cnt, spd, fld]
    return skill_list

def assign_names(first_list, last_list):
    
    r1 = int(np.random.uniform(0, len(first_list)))
    r2 = int(np.random.uniform(0, len(last_list)))
    
    first = first_list[r1]
    last = last_list [r2]
    
    return first, last


In [56]:
## generate teams and rosters

## test roster
team_list = []

## makes name files into lists
f_first = open("first_names.txt", "r")
first_list= [name.replace('\n','') for name in f_first.readlines()]
f_first.close()

f_last = open("last_names.txt", "r")
last_list= [name.replace('\n','').title() for name in f_last.readlines()]
f_last.close()

for i, county in enumerate(county_list):
    mascot = mascot_list[i]
    team_name = f'{county} {mascot}'
    roster = []
    
    for pos in pos_list:

        skill_list = assign_skills()
        first, last = assign_names(first_list, last_list)
        new_player = Player(first, last, pos, team_name, *skill_list)

        roster.append(new_player)
        print(new_player.first, new_player.last, new_player.pos)
        print(new_player.print_skills())
        
    new_team = Team(county, mascot, roster)
    team_list.append(new_team)
    print(new_team.county, new_team.mascot)
    print(new_team.print_record())


Gregory Wilson C
pwr=5.0 cnt=7.0 spd=8.0 fld=2.0
Carl Butler 1B
pwr=5.0 cnt=3.0 spd=2.0 fld=4.0
Adam Flores 2B
pwr=9.0 cnt=7.0 spd=2.0 fld=3.0
Johnny Richardson 3B
pwr=8.0 cnt=1 spd=2.0 fld=7.0
Wayne Rodriguez SS
pwr=3.0 cnt=5.0 spd=7.0 fld=6.0
Matthew Henderson DH
pwr=8.0 cnt=1 spd=1 fld=6.0
Benjamin Gonzalez LF
pwr=8.0 cnt=6.0 spd=7.0 fld=2.0
Jeffrey Hernandez RH
pwr=1.0 cnt=5.0 spd=9 fld=7.0
Willie Brooks CF
pwr=7.0 cnt=4.0 spd=4.0 fld=6.0
Litchfield Leopards
0-0
Russell Martin C
pwr=7.0 cnt=4.0 spd=3.0 fld=6.0
Carl Torres 1B
pwr=6.0 cnt=8.0 spd=3.0 fld=4.0
Terry Hernandez 2B
pwr=8.0 cnt=2.0 spd=1 fld=3.0
Alan Watson 3B
pwr=5.0 cnt=9 spd=5.0 fld=5.0
Jordan Ross SS
pwr=2.0 cnt=7.0 spd=6.0 fld=1.0
Brandon Williams DH
pwr=6.0 cnt=5.0 spd=3.0 fld=5.0
Terry Watson LF
pwr=3.0 cnt=8.0 spd=7.0 fld=5.0
Henry Evans RH
pwr=6.0 cnt=7.0 spd=7.0 fld=5.0
Austin Barnes CF
pwr=9.0 cnt=3.0 spd=2.0 fld=5.0
Hartford Warriors
0-0
Charles Miller C
pwr=4.0 cnt=3.0 spd=3.0 fld=8.0
Timothy Bryant 1B
pwr=7.0

In [57]:
'''# simulate a test game, at-bat by at-bat

test_game = Game(team_list[0], team_list[1])
result, final_rosters = test_game.play_game()

# return final stats on each player
for player in final_rosters[0]:
    print (player.first, player.last, player.pos)
    print (player.print_skills())
    print (player.print_stats(), '\n')
'''

"# simulate a test game, at-bat by at-bat\n\ntest_game = Game(team_list[0], team_list[1])\nresult, final_rosters = test_game.play_game()\n\n# return final stats on each player\nfor player in final_rosters[0]:\n    print (player.first, player.last, player.pos)\n    print (player.print_skills())\n    print (player.print_stats(), '\n')\n"

In [58]:
# simulate full season of 80 games between teams

team_num = len(team_list)
gamedays = 80

for day in range(gamedays):
    print('day = ', day)
    for g in range(team_num//2):
        
        # get team indices so that all teams play each other
        t1 = g
        t2 = (team_num//2) + (g+day)%(team_num//2)
        
        # instantiate game between two teams
        game = Game(team_list[t1], team_list[t2])
        
        # update team records based on game outcome
        if game.play_game(verbose=0)[0]:
            team_list[t1].win += 1 
            team_list[t2].loss += 1
        else:
            team_list[t2].win += 1 
            team_list[t1].loss += 1
        

day =  0
O
S
O
S
O
switch
D
S
H
O
O
K
K
K
S
S
O
switch
K
S
K
K
O
O
K
switch
O
O
H
S
K
O
S
S
O
T
O
switch
O
S
O
K
O
O
O
switch
B
K
K
O
O
K
S
O
switch
O
S
K
O
K
S
T
K
O
switch
K
D
B
O
S
D
O
K
S
O
K
switch
O
O
D
T
D
S
O
S
K
D
T
O
O
switch
H
K
O
D
K
O
S
O
K
switch
K
S
H
B
K
K
D
O
O
K
switch
O
S
K
S
S
D
K
K
D
T
T
D
H
O
O
switch
S
O
O
S
O
K
S
O
B
S
K
switch
K
O
K
O
T
O
K
switch
S
S
K
O
K
S
O
S
K
S
D
O
switch
D
O
O
O
S
H
O
O
S
O
switch
O
K
S
K
K
K
S
D
D
H
O
switch
D
O
D
O
S
B
O
K
O
D
O
switch
O
H
K
H
S
O
K
K
K
switch
H
O
O
O
D
S
K
O
K
switch
O
K
K
K
O
O
switch
O
D
O
O
K
O
O
switch
D
O
B
O
H
D
S
O
O
O
O
switch
D
K
K
O
T
D
O
O
O
switch
K
S
S
O
K
S
S
O
H
K
S
O
switch
O
O
B
K
D
O
O
K
switch
B
K
S
S
O
O
B
O
K
S
K
switch
K
O
T
K
K
K
O
switch
O
S
O
K
S
T
K
K
S
O
switch
K
O
O
B
S
S
S
O
O
S
K
switch
O
O
O
K
O
K
switch
K
O
K
K
B
O
D
O
switch
O
O
O
H
O
O
S
S
O
switch
K
S
S
O
K
K
T
O
O
switch
K
B
O
O
H
S
O
B
K
O
switch
O
K
K
O
K
K
switch
O
K
O
day =  1
K
K
B
S
O
switch
S
O
B
K
O
K
K
H
K
switch
H
B
S
S
S


O
K
O
O
K
switch
O
D
O
O
K
O
D
S
S
O
switch
O
O
K
D
O
K
K
switch
O
B
K
K
K
T
O
O
switch
O
S
O
O
O
S
O
S
K
switch
O
O
K
H
O
O
K
switch
O
O
D
O
D
D
O
T
S
K
O
switch
T
O
S
O
K
day =  11
O
O
K
switch
O
K
S
S
S
O
O
S
O
S
B
O
switch
O
K
O
B
O
B
S
O
O
switch
K
S
S
O
T
K
O
K
O
switch
O
B
O
S
O
O
B
O
K
switch
K
O
K
O
K
O
switch
O
D
T
K
B
K
K
O
K
switch
K
B
O
K
K
K
K
switch
D
D
D
T
O
O
B
S
S
O
O
O
D
O
switch
O
K
K
K
D
B
K
K
switch
K
O
S
K
K
D
O
O
switch
O
O
S
O
T
O
K
O
switch
O
O
K
K
O
S
D
O
switch
O
D
O
K
S
S
D
O
O
K
switch
K
D
K
K
K
K
O
switch
K
H
O
B
K
O
T
B
O
K
switch
K
S
K
K
B
O
O
O
switch
O
H
K
O
K
D
H
O
O
switch
D
D
O
O
D
T
T
O
O
D
H
S
D
D
O
B
K
switch
S
O
O
O
K
O
O
switch
O
B
S
O
O
O
K
S
O
switch
K
O
S
S
T
K
H
S
S
K
H
K
O
switch
O
O
S
K
K
T
O
O
switch
K
O
K
O
K
K
switch
K
T
K
K
O
O
K
switch
B
O
O
O
K
K
S
K
switch
H
K
K
K
O
K
K
switch
H
S
O
K
K
S
K
O
D
K
switch
K
B
O
O
S
T
O
K
O
switch
S
O
B
O
S
K
S
O
H
O
S
S
S
S
K
switch
D
D
K
O
O
O
B
K
O
switch
S
O
O
S
O
S
K
H
O
S
K
switch
K
K
O
O
O
H
O

O
O
K
O
K
S
S
O
switch
K
K
O
B
O
O
B
O
switch
O
S
K
S
O
S
K
K
D
O
switch
H
O
S
D
O
O
K
O
T
O
switch
K
O
O
O
O
H
S
O
switch
O
O
K
H
K
D
K
K
switch
O
S
O
K
K
S
O
H
B
O
switch
O
B
O
O
T
S
T
O
K
S
K
switch
O
O
O
S
B
O
K
S
O
switch
S
S
B
O
T
S
D
S
O
K
K
K
S
O
switch
K
K
O
K
S
S
S
O
K
switch
O
K
B
S
S
S
O
K
O
D
K
switch
O
O
S
O
S
O
O
T
O
switch
K
O
K
K
T
D
O
O
switch
K
S
B
B
K
S
D
O
day =  21
K
H
O
K
switch
D
O
H
D
S
O
K
B
K
D
K
D
O
switch
D
K
D
O
D
O
O
K
K
switch
D
O
O
B
O
O
S
O
O
switch
O
S
K
O
B
O
O
T
O
switch
K
D
O
D
K
O
O
K
switch
K
O
O
O
O
K
switch
S
O
O
H
O
O
B
S
K
S
O
switch
O
O
O
K
O
K
switch
D
D
H
S
S
O
S
O
K
T
T
O
S
O
O
switch
K
S
K
O
S
D
K
T
S
O
O
switch
B
K
O
O
O
S
O
B
K
switch
O
O
D
O
O
O
B
O
switch
K
T
K
O
S
O
K
O
switch
K
S
K
K
B
S
K
S
O
O
switch
K
K
K
O
O
K
switch
K
O
B
O
T
S
O
O
S
O
switch
H
O
O
O
S
S
O
O
T
S
D
K
switch
O
K
O
D
H
D
S
O
S
O
O
switch
O
O
O
K
O
O
switch
S
D
D
O
O
O
H
T
D
O
H
K
K
switch
S
T
K
T
D
K
H
K
S
O
O
O
switch
O
K
B
B
K
K
S
K
O
switch
B
S
T
K
T
S
D
S
O
S

S
S
K
K
O
switch
O
O
O
S
K
S
O
T
K
switch
S
D
O
O
K
K
B
S
O
K
switch
K
O
S
S
D
O
O
S
H
O
K
switch
O
K
O
O
S
O
B
O
switch
K
O
S
O
K
O
O
switch
D
O
O
O
S
H
H
D
B
K
O
O
switch
S
D
O
S
D
O
K
K
K
H
K
switch
O
O
O
O
S
O
O
switch
O
T
H
S
O
H
O
O
O
O
switch
B
S
O
S
S
K
B
K
O
T
K
O
switch
K
O
O
K
S
S
K
K
switch
K
D
O
K
K
H
D
T
K
O
switch
B
S
K
O
S
K
O
O
O
switch
O
O
O
O
O
S
B
D
O
switch
K
B
K
O
O
O
K
switch
K
O
T
O
S
O
K
K
switch
K
O
D
O
K
K
S
K
switch
O
D
O
O
O
O
O
switch
K
O
O
S
K
S
K
K
switch
H
S
B
K
B
O
D
O
O
O
O
switch
O
T
S
D
K
O
day =  31
K
K
K
switch
S
S
O
S
H
O
O
S
S
H
K
D
D
O
K
switch
S
O
B
S
S
K
O
K
K
O
switch
K
O
S
S
O
S
K
S
D
O
S
O
switch
O
K
H
O
O
B
K
O
switch
B
K
S
K
O
K
K
O
switch
K
K
T
B
O
O
O
S
O
switch
S
D
K
K
B
K
O
K
S
O
switch
K
O
O
S
K
H
O
K
switch
O
O
K
D
B
T
O
O
K
switch
S
K
D
B
O
O
O
O
K
switch
K
K
O
K
D
S
S
K
O
switch
D
K
O
K
D
K
O
O
switch
B
O
O
K
K
S
H
O
K
switch
O
K
S
K
K
K
D
S
O
switch
K
K
K
T
S
O
K
S
O
switch
O
S
K
O
K
K
K
switch
S
O
O
S
H
S
K
O
B
H
O
K
switch
O
O

O
O
switch
K
K
O
O
K
H
K
switch
K
O
O
K
B
O
K
switch
O
K
K
K
O
K
switch
K
B
O
O
O
K
K
switch
S
K
S
O
O
S
K
O
S
K
switch
O
B
H
K
K
K
O
O
switch
S
H
K
S
K
O
K
O
B
K
switch
O
O
S
O
S
S
K
O
D
S
S
O
switch
S
K
K
S
B
K
S
K
O
O
switch
S
B
O
O
K
K
K
O
switch
O
K
K
S
D
K
S
K
O
switch
T
K
T
D
K
O
O
O
O
switch
O
O
H
S
S
S
K
O
O
S
H
K
switch
K
K
K
D
O
O
B
H
K
switch
O
K
O
K
K
S
O
switch
S
D
K
K
S
B
B
K
S
O
B
S
K
K
switch
O
O
O
K
O
H
O
switch
S
O
H
O
O
B
O
S
D
K
S
H
D
O
switch
S
O
K
O
H
O
O
K
switch
O
O
K
O
O
S
D
S
O
switch
D
O
S
O
O
O
O
O
switch
K
H
O
O
S
S
O
K
O
switch
K
O
S
B
S
O
O
D
O
K
switch
K
O
O
day =  41
K
S
T
O
T
S
O
switch
O
K
K
S
S
O
S
S
K
O
switch
K
S
O
O
O
K
O
switch
O
S
S
O
O
S
K
O
D
S
T
S
K
switch
O
O
K
D
O
O
O
switch
K
D
K
S
D
K
T
O
O
O
switch
K
S
K
O
K
O
O
switch
S
S
O
O
S
K
K
D
S
K
D
K
switch
O
O
O
O
H
O
O
switch
O
K
K
K
K
O
switch
O
O
O
O
O
K
switch
O
O
O
S
K
K
K
switch
D
O
K
T
O
K
O
K
switch
K
K
S
S
O
O
S
O
K
switch
O
K
O
B
O
O
O
switch
S
K
K
K
T
K
S
K
O
switch
K
B
S
H
O
O
O
H


O
B
O
O
H
S
S
O
K
S
K
switch
O
B
K
S
O
O
S
T
K
O
switch
D
O
O
O
H
K
O
K
switch
S
B
O
K
K
S
B
S
K
K
K
switch
O
O
O
K
K
T
K
switch
O
S
K
S
O
D
O
K
B
O
switch
D
K
O
O
K
O
K
switch
O
S
D
K
S
O
K
D
S
S
K
O
switch
K
S
O
O
D
K
K
O
switch
K
H
O
D
H
H
S
O
B
D
O
D
K
B
O
switch
K
S
S
S
O
D
O
day =  51
O
K
S
D
S
O
switch
D
H
O
K
B
S
B
S
O
K
T
K
O
switch
T
S
O
O
K
K
O
D
K
switch
S
O
K
K
D
B
B
O
K
S
K
switch
O
B
O
B
B
D
O
D
B
H
S
K
K
D
O
switch
O
O
K
K
K
S
K
switch
O
O
K
T
S
B
H
K
T
O
K
switch
O
O
O
K
O
K
switch
T
K
O
B
O
K
O
O
switch
B
K
K
K
S
K
T
K
B
T
S
O
switch
O
K
O
H
B
K
O
O
switch
S
T
D
O
O
B
H
K
K
O
S
K
switch
O
O
D
K
O
O
O
switch
K
K
O
K
K
O
switch
S
K
B
D
K
S
B
O
K
O
K
switch
H
D
B
O
K
D
O
K
K
K
switch
S
O
K
S
K
O
B
O
S
S
S
O
switch
H
S
O
K
O
O
O
S
D
S
H
S
O
switch
O
O
K
O
K
H
D
D
O
switch
S
O
O
O
O
O
T
S
K
switch
S
K
K
O
O
K
S
S
H
O
switch
O
O
O
O
D
T
O
S
O
switch
H
K
K
B
K
O
S
K
K
switch
O
K
S
K
K
O
O
switch
O
S
K
O
D
K
K
O
switch
T
O
S
K
D
O
K
K
O
switch
O
K
O
D
O
T
S
S
S
H
O
B
B
K
swit

K
switch
K
O
O
O
O
O
switch
K
D
O
K
B
K
O
K
switch
K
O
O
O
S
S
O
D
O
switch
S
T
K
S
K
O
O
O
K
switch
O
D
O
S
B
O
O
O
O
switch
H
S
O
B
S
T
O
O
S
O
O
O
switch
K
O
K
K
O
S
K
switch
B
O
S
K
K
O
K
O
switch
O
K
S
K
O
S
S
H
K
K
switch
K
K
K
O
O
K
switch
O
B
H
O
K
K
K
S
O
switch
S
B
K
D
S
K
K
O
B
H
O
O
switch
O
O
S
O
O
K
T
S
K
switch
K
K
K
B
T
K
K
H
K
switch
K
S
O
D
K
D
K
O
K
switch
D
S
O
B
S
S
S
K
O
K
D
S
K
B
K
switch
O
B
B
O
O
K
O
O
switch
O
O
O
O
D
K
S
S
H
K
switch
K
S
O
O
K
S
S
D
S
O
S
O
switch
K
K
H
K
K
O
O
switch
K
H
K
K
S
O
K
O
switch
O
O
K
S
D
O
K
O
switch
O
S
K
T
D
O
O
S
O
K
switch
S
D
O
H
D
O
O
K
O
O
switch
O
S
O
O
O
O
D
S
O
switch
K
O
K
T
O
O
O
switch
B
O
S
K
O
K
S
O
K
switch
S
O
O
H
O
S
B
K
O
S
S
K
switch
O
O
O
O
D
O
S
H
K
switch
B
H
K
K
T
K
B
K
S
O
D
K
switch
S
O
S
O
O
day =  62
D
O
O
B
K
switch
T
O
S
K
O
S
O
B
K
K
switch
O
K
O
K
K
O
switch
H
T
O
K
K
D
H
S
T
O
D
O
O
switch
K
H
K
O
O
O
O
switch
O
O
O
B
O
K
T
O
switch
O
T
O
S
O
O
O
D
O
switch
O
T
H
K
S
O
S
K
S
O
S
O
switch
O
O
S
K
K

O
O
O
S
O
switch
O
O
K
O
O
O
switch
K
O
H
O
H
B
K
D
O
K
switch
D
O
K
K
K
O
S
D
S
D
O
switch
S
O
D
B
K
S
K
O
B
B
O
O
switch
O
K
K
K
T
S
K
O
switch
O
K
O
O
K
O
switch
O
B
S
K
T
T
O
S
O
H
T
O
O
switch
K
O
O
S
S
O
H
B
B
B
H
O
O
switch
H
K
K
K
day =  72
H
S
H
S
O
H
S
O
O
switch
O
K
K
B
K
O
K
switch
O
T
H
O
O
K
O
O
switch
D
K
K
S
S
O
S
K
O
O
switch
O
O
S
K
K
S
H
S
T
O
O
switch
S
K
B
D
K
O
K
K
K
switch
K
B
B
K
K
S
K
O
O
switch
K
H
S
O
B
D
K
T
D
O
D
O
K
switch
H
O
K
K
O
O
K
switch
K
O
K
O
O
K
switch
K
S
K
O
S
K
O
O
switch
O
O
K
S
O
S
T
D
K
S
O
switch
S
K
K
B
S
O
H
K
O
K
switch
O
D
K
B
O
O
O
O
switch
K
H
O
K
B
O
O
O
switch
S
D
T
K
K
O
O
O
K
switch
B
O
S
O
O
S
D
O
O
K
switch
T
O
O
O
S
K
K
D
K
switch
T
O
O
T
S
K
O
O
H
S
K
switch
D
K
O
O
O
O
O
switch
S
K
O
O
K
B
O
S
K
switch
O
S
O
D
B
O
K
O
O
switch
K
K
O
O
S
O
K
switch
O
O
K
D
K
O
K
switch
K
S
H
O
K
O
O
D
B
B
K
switch
O
D
K
K
O
O
K
switch
O
S
O
K
O
O
O
switch
K
O
O
O
S
O
H
O
switch
S
O
O
O
O
O
O
switch
K
H
K
K
S
O
O
O
switch
O
K
K
K
T
D
O
O
switc

In [26]:
# look at full team roster and stats for first team

for player in team_list[0].roster:
    
    print (player.first, player.last)
    print (player.print_skills())
    print (player.print_stats(), '\n')

Richard Robinson
pwr=5.0 cnt=7.0 spd=5.0 fld=4.0
hits=137 walks=26 atbat=426 avg=0.322 rbi=67 hmr=18 

Johnny Wilson
pwr=8.0 cnt=4.0 spd=5.0 fld=1
hits=103 walks=27 atbat=414 avg=0.249 rbi=60 hmr=13 

Nathan Cook
pwr=6.0 cnt=7.0 spd=4.0 fld=3.0
hits=118 walks=24 atbat=407 avg=0.29 rbi=51 hmr=16 

Jack Gonzalez
pwr=8.0 cnt=2.0 spd=3.0 fld=7.0
hits=98 walks=31 atbat=390 avg=0.251 rbi=61 hmr=19 

Dylan Cox
pwr=9.0 cnt=1.0 spd=1 fld=2.0
hits=94 walks=36 atbat=372 avg=0.253 rbi=56 hmr=22 

Joseph Wood
pwr=8.0 cnt=7.0 spd=1 fld=5.0
hits=113 walks=29 atbat=371 avg=0.305 rbi=39 hmr=8 

Jose Bennett
pwr=5.0 cnt=5.0 spd=6.0 fld=6.0
hits=109 walks=27 atbat=363 avg=0.3 rbi=60 hmr=16 

Carl Wilson
pwr=2.0 cnt=1.0 spd=5.0 fld=6.0
hits=110 walks=20 atbat=361 avg=0.305 rbi=47 hmr=12 

Tyler Phillips
pwr=4.0 cnt=6.0 spd=4.0 fld=7.0
hits=103 walks=23 atbat=350 avg=0.294 rbi=71 hmr=17 



In [61]:
# rank all players by stat of interest

def rank_by_stat(stat = 'hmrs', teams = team_list):
    
    all_players = []
    all_stats = []
    for team in team_list:
        for player in team.roster:
            
            all_players.append(player)
            all_stats.append(getattr(player, stat))
            
    sort_idx = np.argsort(all_stats)
    players_sorted = [all_players[i] for i in reversed(sort_idx)]
    
    # return sorted player objects
    return players_sorted

# see attributes in player class for stats to rank by

stat = 'rbis'
ranked_players = rank_by_stat(stat)

for player in ranked_players:
    print ( f'{player.first} {player.first}\n{player.team}'
            f'\n{player.print_skills()}\n{player.print_stats()}\n')
    

Stephen Stephen
Fairfield Founders
pwr=1.0 cnt=8.0 spd=6.0 fld=4.0
hits=156 walks=32 atbat=380 avg=0.411 rbi=89 hmr=14

Paul Paul
Fairfield Founders
pwr=2.0 cnt=5.0 spd=5.0 fld=1.0
hits=114 walks=36 atbat=360 avg=0.317 rbi=84 hmr=15

Raymond Raymond
Tolland Braves
pwr=9.0 cnt=5.0 spd=1 fld=1.0
hits=95 walks=27 atbat=331 avg=0.287 rbi=79 hmr=17

Jesse Jesse
New Haven Sharks
pwr=1.0 cnt=7.0 spd=5.0 fld=5.0
hits=133 walks=28 atbat=349 avg=0.381 rbi=79 hmr=9

Eugene Eugene
New Haven Sharks
pwr=1.0 cnt=7.0 spd=7.0 fld=8.0
hits=131 walks=16 atbat=365 avg=0.359 rbi=78 hmr=11

Henry Henry
Hartford Warriors
pwr=6.0 cnt=7.0 spd=7.0 fld=5.0
hits=112 walks=31 atbat=314 avg=0.357 rbi=76 hmr=16

Kyle Kyle
New London Whalers
pwr=2.0 cnt=6.0 spd=3.0 fld=4.0
hits=131 walks=22 atbat=366 avg=0.358 rbi=76 hmr=16

Jeffrey Jeffrey
Fairfield Founders
pwr=2.0 cnt=4.0 spd=9 fld=1.0
hits=126 walks=31 atbat=378 avg=0.333 rbi=75 hmr=14

Gregory Gregory
Litchfield Leopards
pwr=5.0 cnt=7.0 spd=8.0 fld=2.0
hits=136 